In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
from six.moves import xrange
import tensorflow as tf
import os
import cnn as model
import config
import input_data
import data_producer

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.0003, 'Initial learning rate.')
flags.DEFINE_integer('num_epochs', 400, 'Number of epochs to run trainer.')
flags.DEFINE_integer('batch_size', 32, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')

In [ ]:
IMAGE_SIZE = config.patch_size
CHANNELS = config.channels
NUM_CLASSES = config.num_classes

reduce_lr_epoch_1 = 200
reduce_lr_epoch_2 = 300


TRAIN_FILES = 8
TEST_FILES = 8
DATA_PATH = os.path.join(os.getcwd(),"Data")

In [ ]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    These placeholders are used as inputs by the rest of the model building
    code and will be fed from the downloaded data in the .run() loop, below.
    Args:
    batch_size: The batch size will be baked into both placeholders.
    Returns:
    images_placeholder: Images placeholder.
    labels_placeholder: Labels placeholder.
    is_training: Is_training placeholder.
    learning_rate: Learning_rate placeholder.
    """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=(None, model.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(None))
    is_training = tf.placeholder(dtype=tf.bool)
    learning_rate = tf.placeholder(tf.float32)
    return images_placeholder, labels_placeholder, is_training, learning_rate


In [ ]:
def fill_feed_dict(data_set, batch_size, images_pl, labels_pl, is_training, TF, learning_rate):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
    }
    Args:
    data_set: The set of images and labels, from input_data.read_data_sets()
    batch_size: The batch size will be baked into both placeholders.
    images_pl: The images placeholder, from placeholder_inputs().
    labels_pl: The labels placeholder, from placeholder_inputs().
    is_training: The is_training placeholder, from placeholder_inputs().
    TF: Training or not.
    learning_rate: The learning_rate placeholder, from placeholder_inputs().
    Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
    labels_feed: The next batch label examples.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    images_feed, labels_feed = data_set.next_batch(batch_size)
    feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
      is_training: TF,
      learning_rate: FLAGS.learning_rate,
    }
    return feed_dict, labels_feed

In [ ]:
def do_eval(sess,
            predict,
            images_placeholder,
            labels_placeholder,
            is_training,
            learning_rate,
            data_set):
    """Runs one evaluation against the full epoch of data.
    Args:
    sess: The session in which the model has been trained.
    predict: The Tensor that returns the prediction result of logits.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    is_training: The is_training placeholder.
    learning_rate: The learning_rate placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
    """
    matrix = np.zeros((NUM_CLASSES,NUM_CLASSES))
    # And run one epoch of eval.
    steps_per_epoch = data_set.num_examples // FLAGS.batch_size
    num_rest_examples = data_set.num_examples - steps_per_epoch * FLAGS.batch_size
    for step in xrange(steps_per_epoch):
        feed_dict, labels_feed = fill_feed_dict(data_set,
                                   FLAGS.batch_size,
                                   images_placeholder,
                                   labels_placeholder,
                                   is_training,
                                   False,
                                   learning_rate,)
        prediction = sess.run(predict, feed_dict=feed_dict)

        for i in range(FLAGS.batch_size):
            matrix[int(labels_feed[i])][prediction[i]] += 1
    
    if ( num_rest_examples != 0 ):
        feed_dict, labels_feed = fill_feed_dict(data_set,
                                   num_rest_examples,
                                   images_placeholder,
                                   labels_placeholder,
                                   is_training,
                                   False,
                                   learning_rate,)
        prediction = sess.run(predict, feed_dict=feed_dict)
        for i in range(num_rest_examples):
            matrix[int(labels_feed[i])][prediction[i]] += 1
            
    oa,aa,kappa,prec = calculate_precision(matrix)
    return oa,aa,kappa,prec
 


In [ ]:
def calculate_precision(matrix):
    matrix_sum_col = np.zeros(NUM_CLASSES)
    matrix_sum_row = np.zeros(NUM_CLASSES)
    diagonal_sum = 0
    total_num = 0
    for i in range(NUM_CLASSES):
        diagonal_sum += matrix[i][i]
        for j in range(NUM_CLASSES):
            matrix_sum_col[j] += matrix[i][j]
            matrix_sum_row[i] += matrix[i][j]
            total_num += matrix[i][j]

    oa = diagonal_sum / total_num
    aa = 0
    prec = 'class#\tright\ttest\trate\n'
    for i in range(NUM_CLASSES):
        prec_per_class = matrix[i][i] / matrix_sum_row[i]
        aa += prec_per_class
        prec = prec + '%d\t%d\t%d\t%.5f\n' % (i+1, int(matrix[i][i]), int(matrix_sum_row[i]),prec_per_class)
    aa = aa / NUM_CLASSES

    kappa_temp = 0
    for i in range(NUM_CLASSES):
        kappa_temp += matrix_sum_col[i] * matrix_sum_row[i]
    kappa = (total_num*diagonal_sum-kappa_temp) / (total_num*total_num-kappa_temp)
   
    print('  OA: %.5f' % oa)
    print('  AA: %.5f' % aa)
    print('  kappa: %.5f' % kappa)

    return oa,aa,kappa, prec

In [ ]:
def train_one_epoch(sess,
                    train_op,
                    loss,
                    images_placeholder,
                    labels_placeholder,
                    is_training,
                    learning_rate,
                    data_set):

    steps_per_epoch = data_set.num_examples // FLAGS.batch_size
    total_loss = []
    for step in xrange(steps_per_epoch):
        feed_dict,_ = fill_feed_dict(data_set,
                                   FLAGS.batch_size,
                                   images_placeholder,
                                   labels_placeholder,
                                   is_training,
                                   True,
                                   learning_rate)
        _, loss_value = sess.run([train_op, loss],
                                   feed_dict=feed_dict)
        total_loss.append(loss_value)
    mean_loss = np.mean(total_loss)
    return mean_loss

In [ ]:
def add_DataSet(first,second):
    temp_image = np.concatenate((first.images,second.images),axis=0)
    temp_labels = np.concatenate((first.labels,second.labels),axis=0)
    temp_image = temp_image.reshape(temp_image.shape[0],IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
    temp_image = np.transpose(temp_image,(0,3,1,2))
    temp_labels = np.transpose(temp_labels)
    return input_data.DataSet(temp_image,temp_labels)

In [ ]:
def count_trainable_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value
        total_parameters += variable_parametes
    print("  Total training params: %.1fK" % (total_parameters / 1e3))

In [ ]:
def concat_training_and_testing_dataset():
    """Concatenating all the training and test mat files"""
    for i in range(TRAIN_FILES):
        data_sets = input_data.read_data_sets(os.path.join(DATA_PATH, config.dataset+'_Train_'+str(IMAGE_SIZE)+'_'+str(i+1)+'.mat'), 'train')
        if(i==0):
            Training_data = data_sets
            continue
        else:
            Training_data = add_DataSet(Training_data,data_sets)
            
    for i in range(TEST_FILES):
        data_sets = input_data.read_data_sets(os.path.join(DATA_PATH, config.dataset+'_Test_'+str(IMAGE_SIZE)+'_'+str(i+1)+'.mat'),'test')
        if(i==0):
            Test_data = data_sets
            continue
        else:
            Test_data = add_DataSet(Test_data,data_sets)
    return Training_data, Test_data

In [ ]:
def main(_):
    """Train and evaluate model for a number of times."""
        
    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
        # Generate placeholders for the images, labels, is_training and learning_rate.
        images_placeholder, labels_placeholder, is_training, learning_rate = placeholder_inputs(FLAGS.batch_size)

        # Build a Graph that computes predictions from the inference model.
        logits = model.inference(images_placeholder, is_training)

        # Add to the Graph the Ops for loss calculation.
        loss = model.loss(logits, labels_placeholder)

        # Add to the Graph the Ops that calculate and apply gradients.
        train_op = model.training(loss, learning_rate)

        # Add the Op to calculate prediction result of logits..
        predict = model.predicting(logits)

        sess = tf.Session()
        
        saver = tf.train.Saver(max_to_keep=1000)

        training_times = 10
        for times_counter in xrange(training_times):
            data_producer.create_dataset()
            Training_data, Test_data = concat_training_and_testing_dataset()
            

            init = tf.global_variables_initializer()
            sess.run(init)
            count_trainable_params()
            FLAGS.learning_rate = 0.0003
            duration1 = 0
            train_duration = 0
            test_duration = 0
            train_oa = 0
            train_aa = 0
            train_kappa = 0
            train_loss = 0
            train_epoch = 0
            
            # Start the training loop.
            for epoch in xrange(FLAGS.num_epochs):
                train_start_time = time.time()

                if epoch == reduce_lr_epoch_1 or epoch == reduce_lr_epoch_2:
                    FLAGS.learning_rate = FLAGS.learning_rate / 10
                    #print("Decrease learning rate, new lr = %f" % FLAGS.learning_rate)
                
                one_epoch_loss = train_one_epoch(sess,
                                                 train_op,
                                                 loss,
                                                 images_placeholder,
                                                 labels_placeholder,
                                                 is_training,
                                                 learning_rate,
                                                 Training_data)
                duration1 += (time.time() - train_start_time)
                

                if (epoch + 1) % 20 == 0 or (epoch + 1) == FLAGS.num_epochs:
                    test_start_time = time.time()
                    oa,aa,kappa,prec = do_eval(sess,
                                        predict,
                                        images_placeholder,
                                        labels_placeholder,
                                        is_training,
                                        learning_rate,
                                        Test_data)
                    duration2 = (time.time() - test_start_time)

                    if oa > train_oa:
                        train_oa = oa
                        train_aa = aa
                        train_kappa = kappa
                        train_duration = duration1
                        test_duration = duration2
                        train_loss = one_epoch_loss
                        train_epoch = epoch
                        train_prec = prec
                        saver.save(sess, DATA_PATH+'/record/'+config.dataset+'/cnn_t' + str(times_counter) + '-' + str(IMAGE_SIZE)+'X'+str(IMAGE_SIZE)+'.ckpt', global_step=epoch)

            print('-------------------Training %d-----------------' % times_counter)
            print('  OA: %.5f' % train_oa)
            print('  AA: %.5f' % train_aa)
            print('  kappa: %.5f' % train_kappa)
            print('  Epoch: %d' % train_epoch)
            print('  Training time spent: %.4f s' % train_duration)
            print('  Testing time spent: %.4f s' % test_duration)
            print('  Loss: %.4f' % train_loss)
            #print(prec)
            

In [ ]:
if __name__ == '__main__':
    tf.app.run()